In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv1D, Activation, MaxPooling2D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
import time

In [2]:
from pygameMultibot import GameWindow

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
Python version 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [35]:
class Warehouse(Env):
    #metadata = {'render.modes' : ['human']}
    def __init__(self):
        self.pygame = GameWindow(0,2)
        self.action_space = Discrete(64)
        self.observation_space = Box(low=0, high=6, shape=(11,11,2))

    def reset(self):
        del self.pygame
        self.pygame = GameWindow(0,2)
        obs = self.pygame.observe()
        return obs

    def step(self, action):
        self.pygame.action(action)
        obs = self.pygame.observe()
        reward = self.pygame.evaluate()
        done = self.pygame.is_done()
        # print(obs)
        return obs, reward, done, {}

    def render(self, mode="human", close=False):
        time.sleep(0.05)
        self.pygame.view1()

In [36]:
env = Warehouse()

In [5]:
states = env.observation_space.shape
actions = env.action_space.n

In [6]:
env.observation_space.sample()

array([[2.4452944 , 3.5383372 , 3.6035469 , 4.462473  , 4.7858386 ,
        4.437677  , 4.002102  , 3.9663677 , 1.8884623 ],
       [1.4942323 , 3.5961027 , 2.7873087 , 3.0633903 , 0.5225278 ,
        4.082532  , 0.48847166, 4.8141127 , 2.20195   ],
       [0.53599477, 1.6767579 , 2.6987495 , 1.0686494 , 4.5410066 ,
        1.1085467 , 1.4988247 , 2.6489668 , 0.46038052],
       [2.3357296 , 4.2089124 , 4.606848  , 3.569118  , 1.8193121 ,
        2.4803622 , 4.672733  , 1.8382236 , 2.6241663 ],
       [4.222928  , 3.6386456 , 3.148463  , 3.7675834 , 3.7612913 ,
        1.9249065 , 0.32616535, 4.7445188 , 2.619789  ],
       [1.4045244 , 3.2305334 , 1.9905653 , 0.70471156, 4.2164307 ,
        0.29518104, 1.1506933 , 0.10499869, 3.9615219 ],
       [4.505272  , 1.9112425 , 2.8949535 , 1.0292412 , 1.9222237 ,
        4.6203556 , 0.17338055, 0.75171185, 2.8495283 ],
       [3.1573608 , 2.0951505 , 4.9722805 , 1.8876575 , 3.6862655 ,
        2.5208263 , 0.18214294, 3.060203  , 2.200791  ],


In [7]:
def build_model():
    model = Sequential()
    model.add(tf.keras.layers.Reshape((11,11,2), input_shape=(1,11,11,2)))
    model.add(Conv2D(32, (3,3),strides=(1, 1),activation='relu'))  # OBSERVATION_SPACE_VALUES = (10, 10, 3) a 10x10 RGB image.
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3),strides=(1, 1),activation='relu'))
    model.add(Conv2D(64, (3,3),strides=(1, 1),activation='relu'))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.2))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(512,activation='relu'))

    model.add(Dense(actions, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
    return model

In [13]:
del model

In [14]:
model = build_model()

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 9, 9, 1)           0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 32)          320       
                                                                 
 conv2d_4 (Conv2D)           (None, 5, 5, 64)          18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               295424    
                                                                 
 dense_3 (Dense)             (None, 25)               

In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [17]:
def build_agent(model, actions):
#     policy = EpsGreedyQPolicy()
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=500000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10000, target_model_update=1e-2, enable_double_dqn=True)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=0.001), metrics=['mae'])

D:\anaconda\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [33]:
dqn.compile(Adam(lr=0.000007), metrics=['mae'])
history=dqn.fit(env, nb_steps=1000000, visualize=True, verbose=1)

Training for 1000000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 20s 2ms/step - reward: -0.0879
10 episodes - episode_reward: -87.910 [-186.800, 36.500]

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 147s 15ms/step - reward: 0.0440
10 episodes - episode_reward: 43.980 [17.800, 54.100] - loss: 0.031 - mae: 5.181 - mean_q: 5.679 - mean_eps: 0.100

Interval 3 (20000 steps performed)
 1714/10000 [====>.........................] - ETA: 1:56 - reward: 0.0541done, took 190.959 seconds


In [38]:
_ = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: 87.100, steps: 1000


In [33]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [26]:
dqn.load_weights('dqn_weights.h5f')

In [25]:
dqn.save_weights('dqn_weightsCNN.h5f', overwrite=True)

In [28]:
dqn.load_weights('dqn_weightsCNN.h5f')

In [23]:
dqn.save_weights('dqn_weightsTemp.h5f', overwrite=True)

In [19]:
dqn.load_weights('dqn_weightsTemp.h5f')

In [23]:
dqn.save_weights('dqn_weightsTemp1.h5f', overwrite=True)

In [20]:
dqn.load_weights('dqn_weightsTemp1.h5f')